In [1]:
import pandas as pd
import datetime as datetime
import numpy as np
import copy
from RSU_functions import * 

# If you want to change your marginal income tax (above 72k of taxable income, it is at 41%). Not really needed
tax_info_dict = {'TMI_IR': 0.3, 'cotisation': 0.172, 'flat_tax_plus_value': 0.3, 'deductible_csg':0.0680}

# Put here all sales and vesting event from EquatePlus
Please put all event until end of 2024, including sales / vesting from before (other years). If you have sales in 2023 or 2024 either rely on the built in function or please du check yourself how you reported them before. 

If you sold everything you had at one point, before 2024 then you can put only vesting and transaction after this big sale event.

If you sold everything in 2024, please put all information in 2024 and even before if you want the CSG rebate computation, if you don't, you may start with all events starting after this "big sales". Please note this CSG rebate should be prefilled, so it is only to check.

If you did not sell everything, please find out which stock you declared selling in your previous declaration (for stock sold before this year of 2024) and add the amount sold to the vesting event as amount_already_declared

For sales, you need to add the stock unit price. In equate plus, go to the bibliotheque menu, then transaction. For sales, click on the date which is clickable and then report "prix de vente" IN USD (VERY IMPORTANT).

For Macron law, only stock with grant date later than the 25th of june 2020 are Macron 3. Indeed (ridiculously) the date of interest for the macron law 3 to kick in is not the grant date, but the date of the AG autorising the grant. Sadly all of our grants coming before the June 25th 2020 are NOT macron law eligible. Should you decide to have your (wrong) interpretation of the law, it should work though.

If you sold more than 300k this year, please note that the macron law 1 or 2 differs. This notebook is not handling it correctly.

In [2]:

vesting_event = [
    # Each row for a vesting. If you have more vesting, please copy the full line with the comma at the end, and then put your value on each row. 
    # You should fill just the date and the amount. Please let macron_law_id at 0, we do not have newer RSU. If you decide 
    # to have your personnal interpretation of the law, or have non-criteo RSU, please put 3 instead.
    {'date':datetime.datetime(2019,12,13),'amount':225,'macron_law_id':0, 'event':'vesting', 'amount_already_declared':225},
    {'date':datetime.datetime(2020,3,13),'amount':28,'macron_law_id':0, 'event':'vesting', 'amount_already_declared':28},
    {'date':datetime.datetime(2020,6,15),'amount':29,'macron_law_id':0, 'event':'vesting', 'amount_already_declared':29},
    {'date':datetime.datetime(2020,9,14),'amount':28,'macron_law_id':0, 'event':'vesting', 'amount_already_declared':28},
    {'date':datetime.datetime(2020,12,14),'amount':28,'macron_law_id':0, 'event':'vesting', 'amount_already_declared':28},
    # Amount sold is only for vested stock that you sold and declared in the years before. Here the previous stock was sold the 16 of december
    # so it is the last that I am filling. It is not necessary to fill it if you sold everything you had at your last sale event last year
    # here it should therefore do nothing because in this example, all stocks were sold. In case of doubt, just fill it.
    # When filled, these stocks are sold in priority over all else at the next transaction.
    # If I find stocks that were supposedly declared in 2023 that are sold in 2024, I will raise an error and ask you to check what you wrote. 
    {'date':datetime.datetime(2023,3,15),'amount':28,'macron_law_id':0, 'event':'vesting'},
    {'date':datetime.datetime(2023,6,14),'amount':28,'macron_law_id':0, 'event':'vesting'},
    {'date':datetime.datetime(2023,7,26),'amount':600,'macron_law_id':0, 'event':'vesting'},
    {'date':datetime.datetime(2023,9,13),'amount':28,'macron_law_id':0, 'event':'vesting'},
    {'date':datetime.datetime(2023,10,25),'amount':75,'macron_law_id':0, 'event':'vesting'},
    {'date':datetime.datetime(2023,12,13),'amount':28,'macron_law_id':0, 'event':'vesting'},
    {'date':datetime.datetime(2024,1,25),'amount':75,'macron_law_id':0, 'event':'vesting'},
    {'date':datetime.datetime(2024,4,25),'amount':75,'macron_law_id':0, 'event':'vesting'},
    {'date':datetime.datetime(2024,6,22),'amount':250,'macron_law_id':0, 'event':'vesting'},
    {'date':datetime.datetime(2024,7,25),'amount':75,'macron_law_id':0, 'event':'vesting'},
    {'date':datetime.datetime(2024,9,22),'amount':312,'macron_law_id':0, 'event':'vesting'},
    #     I PUT macron law id 3 for the last vesting taht is sold (the next one in december is not sold) to show you how to put it
    {'date':datetime.datetime(2024,10,25),'amount':75,'macron_law_id':3, 'event':'vesting'},
    {'date':datetime.datetime(2024,12,22),'amount':313,'macron_law_id':0, 'event':'vesting'}
]

selling_event = [
    # Each row for a sale. If you have more sale, please copy the full line with the comma at the end, and then put your value on each row. 
    # You should just fill the date, amount, and stock_unit_price_USD, and total_fee_dollars 
    {'date':datetime.datetime(2020,8,3),'amount':282, 'event':'sale', 'stock_unit_price_USD':13.62, 'total_fee_dollars':38.5},
    {'date':datetime.datetime(2020,12,16),'amount':56, 'event':'sale', 'stock_unit_price_USD':18.80, 'total_fee_dollars':38.5},
    {'date':datetime.datetime(2023,3,16),'amount':28, 'event':'sale', 'stock_unit_price_USD':33.37, 'total_fee_dollars':38.5},
    {'date':datetime.datetime(2023,8,6),'amount':628, 'event':'sale', 'stock_unit_price_USD':39.18, 'total_fee_dollars':108.26},
    {'date':datetime.datetime(2023,9,14),'amount':28, 'event':'sale', 'stock_unit_price_USD':36.72, 'total_fee_dollars':38.5},
    {'date':datetime.datetime(2023,11,5),'amount':75, 'event':'sale', 'stock_unit_price_USD':40.57, 'total_fee_dollars':38.5},
    {'date':datetime.datetime(2023,12,16),'amount':28, 'event':'sale', 'stock_unit_price_USD':40.71, 'total_fee_dollars':38.5},
    {'date':datetime.datetime(2024,5,6),'amount':50, 'event':'sale', 'stock_unit_price_USD':26.00, 'total_fee_dollars':38.5},
    {'date':datetime.datetime(2024,8,8),'amount':30, 'event':'sale', 'stock_unit_price_USD':26.821, 'total_fee_dollars':65.71},
    {'date':datetime.datetime(2024,8,10),'amount':75, 'event':'sale', 'stock_unit_price_USD':27.478, 'total_fee_dollars':93.7},
    {'date':datetime.datetime(2024,8,15),'amount':320, 'event':'sale', 'stock_unit_price_USD':27.478, 'total_fee_dollars':93.7},
    {'date':datetime.datetime(2024,11,3),'amount':387, 'event':'sale', 'stock_unit_price_USD':24.845, 'total_fee_dollars':47.11},

]

In [3]:
recap = '''## Selling events recap
| date | amount | event | price (\$) | fee (\$) | total (\$) |
|------|--------|-------|-------|-----|-------|
'''

total_amount = 0
for ev in sorted(selling_event, key = lambda k: k['date']):
    if ev['date'].year == 2024:
        total_amount += ev['amount'] * ev['stock_unit_price_USD']
        recap += '''| {} | {} | {} | {} | {} | {} |\n'''.format(
            ev['date'].strftime('%d/%m/%Y'),
            ev['amount'],
            ev['event'],
            ev['stock_unit_price_USD'],
            ev['total_fee_dollars'],
            np.round(ev['amount'] * ev['stock_unit_price_USD'], 2))


recap += '\n'
recap += 'Your **total sales** amount in dollars in 2024 is equal to **{} \$**\n'.format(np.round(total_amount, 2))

from IPython.display import display_markdown

display_markdown(recap, raw=True)


## Selling events recap
| date | amount | event | price (\$) | fee (\$) | total (\$) |
|------|--------|-------|-------|-----|-------|
| 06/05/2024 | 50 | sale | 26.0 | 38.5 | 1300.0 |
| 08/08/2024 | 30 | sale | 26.821 | 65.71 | 804.63 |
| 10/08/2024 | 75 | sale | 27.478 | 93.7 | 2060.85 |
| 15/08/2024 | 320 | sale | 27.478 | 93.7 | 8792.96 |
| 03/11/2024 | 387 | sale | 24.845 | 47.11 | 9615.02 |

Your **total sales** amount in dollars in 2024 is equal to **22573.46 \$**


In [4]:
recap = '''## Vesting events recap

| date | amount | event | macron | remaining |
|------|--------|-------|-------|--|
'''

for ev in sorted(vesting_event, key = lambda k: k['date']):
    recap += '''| {} | {} | {} | {} | {} |\n'''.format(
        ev['date'].strftime('%d/%m/%Y'),
        ev['amount'],
        ev['event'],
        ev['macron_law_id'],
        ev['amount']
    )

display_markdown(recap, raw=True)

## Vesting events recap

| date | amount | event | macron | remaining |
|------|--------|-------|-------|--|
| 13/12/2019 | 225 | vesting | 0 | 225 |
| 13/03/2020 | 28 | vesting | 0 | 28 |
| 15/06/2020 | 29 | vesting | 0 | 29 |
| 14/09/2020 | 28 | vesting | 0 | 28 |
| 14/12/2020 | 28 | vesting | 0 | 28 |
| 15/03/2023 | 28 | vesting | 0 | 28 |
| 14/06/2023 | 28 | vesting | 0 | 28 |
| 26/07/2023 | 600 | vesting | 0 | 600 |
| 13/09/2023 | 28 | vesting | 0 | 28 |
| 25/10/2023 | 75 | vesting | 0 | 75 |
| 13/12/2023 | 28 | vesting | 0 | 28 |
| 25/01/2024 | 75 | vesting | 0 | 75 |
| 25/04/2024 | 75 | vesting | 0 | 75 |
| 22/06/2024 | 250 | vesting | 0 | 250 |
| 25/07/2024 | 75 | vesting | 0 | 75 |
| 22/09/2024 | 312 | vesting | 0 | 312 |
| 25/10/2024 | 75 | vesting | 3 | 75 |
| 22/12/2024 | 313 | vesting | 0 | 313 |


# Computation occurs here, please just run these cells

In [5]:
all_events_sorted = sorted(vesting_event + selling_event, key = lambda k: k['date'])

portfolio = {}
portfolio['available_stock'] = []
portfolio['sale_event_recap'] = []
for event in all_events_sorted:
    if event['event'] == 'vesting':
        portfolio['available_stock'].append(copy.deepcopy(event))
    if event['event'] == 'sale':
        available_stock_after_transaction, sale_event_recap = get_sales_result(event, portfolio,
                                                                               matching_method = get_sale_order_from_optionality)
        portfolio['available_stock'] = available_stock_after_transaction
        portfolio['sale_event_recap'].append(sale_event_recap)
    if event['date'].year == 2024:
        sum_stock_supposed_to_be_sold = 0
        for stock in portfolio['available_stock']:
            sum_stock_supposed_to_be_sold += stock.get('amount_already_declared', 0)
        if sum_stock_supposed_to_be_sold> 0:
            raise Exception(
            '''
            you have stock that you already declared in the previous years that are getting sold in 2024. Please recheck
            your inputs. If you declared them, they should have been sold in the past.            
            ''')
        

In [6]:
recap = '''## Unused stocks after 2024 transactions

These stocks should still be on your EquatePlus account if you haven't sold in 2025 yet.

| date | amount | event | macron | remaining |
|------|--------|-------|-------|--|
'''

for ev in sorted(available_stock_after_transaction, key = lambda k: k['date']):
    recap += '''| {} | {} | {} | {} | {} |\n'''.format(
        ev['date'].strftime('%d/%m/%Y'),
        ev['amount'],
        ev['event'],
        ev['macron_law_id'],
        ev['amount']
    )

display_markdown(recap, raw=True)

## Unused stocks after 2024 transactions

These stocks should still be on your EquatePlus account if you haven't sold in 2025 yet.

| date | amount | event | macron | remaining |
|------|--------|-------|-------|--|
| 22/12/2024 | 313 | vesting | 0 | 313 |


# Results to put in your taxation returns

The following cells prints all transaction for the form **No 2074**.

For the cell **511**, you can name it `Action Criteo et Computershare`.

Vesting amount with moins value, rebate etc... are for debugging purposes, please look if they make sense.

To explain quickly what we do here, for each sale, you are taxed on your vesting amount (the value of the stock the day you vest them) as if it was income and with the 30% flat tax for any capital gain on top of the vesting amount.

TAX = `vesting gain` x `progressive income tax rate` + `capital gain` x `flat tax rate` + `total gain` x `social taxes rate`

which is approx:

TAX = `vesting gain` x 30% + `capital gain` x 30% + `total gain` x 17,2%

In order to avoid to tax you on money you do not have, if you have a capital loss, instead of declaring it as a capital loss, it is removed from your vesting amount and in this case, your selling amount(money you get from your sales) becomes your vesting amount.

Then, because this is 🇫🇷 France, we created tax rebate that depends on if the stock was granted to you after the `25th of june 2020`
* after `25th of june 2020`: you only pay income tax on `50%` of the vesting gain because you fall under the **Macron 3 regime**
* before `25th of june 2020`: it also depends if you have kept the stock **2 years after the vesting**. You gain the `50% rebate` after 2 years - `65%` after 8 years

There is a specificity if you sell more than **300k**. This case is **NOT** well handled in this notebook and it won't until.

This explains all the complex computation below.

In [7]:
tax = 0
Capital_gain = 0
vesting_gain_before_rebate = 0
rebate_amount_macron_3 = 0
rebate_amount_from_detention = 0
total_selling_price = 0
csg_rebate = 0
Capital_gain_2023 = 0
rebate_amount_2023 = 0
vesting_gain_before_rebate_2023 = 0

for event in portfolio['sale_event_recap']:
    for ev in event:            
        if ev['date de la cession (513)'].year ==2024:
            tax +=ev['tax']
            total_selling_price += ev['prix de cession net (518)']
            vesting_gain_before_rebate += ev['vesting_amount_with_moins_value']

            if ev['macron_law_id'] == 3:
                rebate_amount_macron_3 += ev['rebate_with_moins_value']
            else:
                rebate_amount_from_detention += ev['rebate_with_moins_value']

            if ev['resultat'] > 0:
                Capital_gain += ev['resultat']
            
        if ev['date de la cession (513)'].year == 2023:
            # here it is to compute CSG than can be dedudected from last year, box 6DE. Only what is 
            # subject to IR can be deducted, hence only the  vesting amount after rebate and moins value
            # will be deductible
            if ev['resultat'] > 0:
                Capital_gain_2023 += ev['resultat']

            rebate_amount_2023 += ev['rebate_with_moins_value']
            vesting_gain_before_rebate_2023 += ev['vesting_amount_with_moins_value']


In [8]:
print_transactions(portfolio, 2024, True)

|   | Sell |
| -- | -- |
| **(511)** Titre | Criteo actions |
| **(513)** Date de cession | 15/08/2024 |
| **(514)** Valeur unitaire de cession | 24.96 |
| **(515)** Nombre de titres cédés | 75 |
| **(516)** Montant total | 1872 |
| **(517)** Frais de cession | 42.55 |
| **(518)** Prix de cession net | 1829 |
| **(520)** Prix unitaire d'acquisition | 23.53 |
| **(521 et 523)** Prix d'acquisition global | 1764.67 |
| **(524)** Résultat | 64.41 |


# For debugging / your interest only

The following cell shows all transaction with a loss.

You don't have to report them in the plus or moins value, but they have been used to compute values in form **2042 C** and are shown here if you want to check that there isn't any mistake or understand better the process.

In [9]:
print_transactions(portfolio, 2024, False)

## (debug only) Transactions with a loss

|   | Sell | Sell | Sell | Sell | Sell | Sell | Sell | Sell |
| -- | -- | -- | -- | -- | -- | -- | -- | -- |
| **(511)** Titre | Criteo actions | Criteo actions | Criteo actions | Criteo actions | Criteo actions | Criteo actions | Criteo actions | Criteo actions |
| **(513)** Date de cession | 06/05/2024 | 08/08/2024 | 10/08/2024 | 10/08/2024 | 10/08/2024 | 15/08/2024 | 03/11/2024 | 03/11/2024 |
| **(514)** Valeur unitaire de cession | 24.13 | 24.54 | 25.15 | 25.15 | 25.15 | 24.96 | 22.79 | 22.79 |
| **(515)** Nombre de titres cédés | 50 | 30 | 45 | 25 | 5 | 245 | 75 | 312 |
| **(516)** Montant total | 1206.38 | 736.17 | 1131.82 | 628.79 | 125.76 | 6113.99 | 1708.89 | 7108.99 |
| **(517)** Frais de cession | 35.73 | 60.12 | 28.59 | 28.59 | 28.59 | 42.55 | 21.6 | 21.6 |
| **(518)** Prix de cession net | 1170.66 | 676.05 | 1103.23 | 600.2 | 97.17 | 6071.44 | 1687.29 | 7087.39 |
| **(520)** Prix unitaire d'acquisition | 34.18 | 41.5 | 41.5 | 34.18 | 33.55 | 33.55 | 38.08 | 38.03 |
| **(521 et 523)** Prix d'acquisition global | 1708.96 | 1244.95 | 1867.43 | 854.48 | 167.75 | 8219.94 | 2855.89 | 11866.61 |
| **(524)** Résultat | -538.3 | -568.91 | -764.2 | -254.28 | -70.59 | -2148.51 | -1168.6 | -4779.22 |


Normally here you should have automatically computed the sum of "plus value", and it should be equal to the value printed in the cell below.

For all transaction resulting in a moins value, I removed the moins value from the vesting price to redue the taxes

In [10]:
display_markdown(f'''# Capital gain

The total capital gain is **{np.round(Capital_gain,2)} €**''', raw=True)


# Capital gain

The total capital gain is **64.41 €**

In [11]:
vesting_gain_after_rebate_and_moins_value = vesting_gain_before_rebate - rebate_amount_macron_3 - rebate_amount_from_detention
total_tax_to_pay_approx = vesting_gain_after_rebate_and_moins_value * (tax_info_dict['TMI_IR']) + vesting_gain_before_rebate * tax_info_dict[
        'cotisation'] + Capital_gain * tax_info_dict[
                     'flat_tax_plus_value']

display_markdown("TAX ~= (TMI) `{}` x `{}` + (Social taxes) `{}` x `{}` + (Capital gain) `{}` x `{}`".format(
    np.round(vesting_gain_after_rebate_and_moins_value, 2),
    tax_info_dict['TMI_IR'],
    np.round(vesting_gain_before_rebate, 2),
    tax_info_dict['cotisation'],
    np.round(Capital_gain, 2),
    tax_info_dict['flat_tax_plus_value']),
                 raw = True)


TAX ~= (TMI) `19671.0` x `0.3` + (Social taxes) `20525.44` x `0.172` + (Capital gain) `64.41` x `0.3`

In [12]:
display_markdown(f'## Form 2042C', raw = True)

if vesting_gain_after_rebate_and_moins_value > 300000:
    display_markdown(f'**1TT** with {np.round(vesting_gain_after_rebate_and_moins_value - 300000, 2)}', raw = True)

display_markdown(f'**1TZ** with {np.round(vesting_gain_after_rebate_and_moins_value, 2)} €', raw = True)
display_markdown(f'**1UZ** abbatement pour duree de detention, the rebate in euro {np.round(rebate_amount_from_detention, 2)} €', raw = True)
display_markdown(f'**1WZ** abattement de 50%, the rebate in euro {np.round(rebate_amount_macron_3, 2)} €', raw = True)

display_markdown(f'**3VG** with capital gain {np.round(Capital_gain, 2)} €', raw = True)
display_markdown(f'**3VH** with capital loss **0 €** except if you have other sources of capital losses. You put **0 €** because you are removing these losses from the **1TZ** field.', raw = True)

display_markdown(f'You sold for **{np.round(total_selling_price,2)}** € in stocks.', raw = True)
display_markdown(f'You will pay around  **{np.round(total_tax_to_pay_approx ,2)}** € this amount of tax.', raw = True)

## Form 2042C

**1TZ** with 19671.0 €

**1UZ** abbatement pour duree de detention, the rebate in euro 0.0 €

**1WZ** abattement de 50%, the rebate in euro 854.45 €

**3VG** with capital gain 64.41 €

**3VH** with capital loss **0 €** except if you have other sources of capital losses. You put **0 €** because you are removing these losses from the **1TZ** field.

You sold for **20322.49** € in stocks.

You will pay around  **9451.0** € this amount of tax.

# CSG deduction from previous year (2023)

WARNING WE CHANGED THE WAY TO COMPENSATE BETWEEN PLUS OR MOINS VALUE SO YOU MAY HAVE SMALL CHANGES HERE

If pre filled, keep the pre filled version

If you have your previous tax filling, it should be equal to your **1TZ** case of last year times the deductible CSG, which is at **6.8%**. If you don't, you can use this, should work. It is very likely to be pre filled though. Still there in case of need


In [13]:
if vesting_gain_before_rebate_2023 ==0:
    print('sorry, no CSG to be deducted for this year!')
    vesting_gain_after_rebate_and_moins_value_2023 =0
    deductible_csg = vesting_gain_before_rebate_2023 * tax_info_dict['deductible_csg']
else:
    vesting_gain_after_rebate_and_moins_value_2023 = vesting_gain_before_rebate_2023 - rebate_amount_2023
    deductible_csg = vesting_gain_after_rebate_and_moins_value_2023 * tax_info_dict['deductible_csg']
   

In [14]:
print('fill box 6DE', deductible_csg)

fill box 6DE 1551.521097080306
